<a href="https://colab.research.google.com/github/Hammadtahir7/Card-Fraud-Detection/blob/master/CCFD_1DCNN_BCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [3]:
import io
import pandas as pd
df2 = pd.read_csv(io.BytesIO(uploaded['BCDD.csv']),header=None)
df2.head()

,0,1,2,3,4,5,6,7
0,1.0,35.0,1.0,11753.0,2.0,4.0,38.0,1.0
1,1.0,24.0,1.0,5835.0,7.0,2.0,68.0,1.0
2,1.0,10.0,1.0,9020.0,11.0,14.0,63.0,1.0
3,1.0,39.0,1.0,9425.0,7.0,0.0,73.0,1.0
4,1.0,17.0,1.0,15672.0,42.0,0.0,43.0,1.0


In [0]:
features =df2.drop(df2.columns[7],axis=1)
labels=df2[df2.columns[7]]

In [5]:
import numpy as np
unique_elements, counts_elements = np.unique(labels, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[0.e+00 1.e+00]
 [5.e+04 5.e+04]]


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
features = scaler.fit_transform(features)
print(features[0])

X_train, X_test, y_train, y_test=train_test_split(features,labels,test_size=0.2, random_state=42)

#X_train=X_train.astype('float32')
#X_test=X_test.astype('float32')
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

[-0.83985099  0.71924274 -0.18851982  0.79273848 -1.15437249 -0.06768473
  1.35873136]
(80000, 7)
(20000, 7)
(80000,)
(20000,)


In [7]:
X_train=X_train.reshape(80000,1,7)
X_test=X_test.reshape(20000,1,7)
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [8]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [9]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1,7)))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [10]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_m,precision_m, recall_m])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
#train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)




Train on 80000 samples, validate on 20000 samples
Epoch 1/10





80000/80000 [==============================] - 6s 72us/step - loss: 0.2760 - acc: 0.8847 - f1_m: 0.8842 - precision_m: 0.8667 - recall_m: 0.9092 - val_loss: 0.2629 - val_acc: 0.8875 - val_f1_m: 0.8866 - val_precision_m: 0.8733 - val_recall_m: 0.9074
Epoch 2/10
80000/80000 [==============================] - 5s 63us/step - loss: 0.2645 - acc: 0.8879 - f1_m: 0.8873 - precision_m: 0.8708 - recall_m: 0.9110 - val_loss: 0.2652 - val_acc: 0.8876 - val_f1_m: 0.8877 - val_precision_m: 0.8645 - val_recall_m: 0.9193
Epoch 3/10
80000/80000 [==============================] - 5s 62us/step - loss: 0.2626 - acc: 0.8881 - f1_m: 0.8878 - precision_m: 0.8700 - recall_m: 0.9127 - val_loss: 0.2717 - val_acc: 0.8868 - val_f1_m: 0.8860 - val_precision_m: 0.8720 - val_recall_m: 0.9075
Epoch 4/10
80000/80000 [==============================] - 5s 63us/step - loss: 0.2620 - acc: 0.8883 - f1_m: 0.8881 - precision_m: 0.8706 - recall_m: 0.9129 - v

In [12]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)


print("loss: "+"{:.2%}".format(loss));
print("acc: "+"{:.2%}".format(accuracy));
print("f1: "+"{:.2%}".format(f1_score));
print("precision: "+"{:.2%}".format(precision));
print("recall: "+"{:.2%}".format(recall));

loss: 27.76%
acc: 88.85%
f1: 88.85%
precision: 86.75%
recall: 91.76%


In [0]:
prediction=model.predict(X_test)

In [14]:
for i in range(len(y_test)):
  if(y_test[i]==1):
    print(i)
    print(y_test[i])
    print(prediction[i])
    print("-----")

2
1.0
[0.85249925]
-----
10
1.0
[0.9667064]
-----
11
1.0
[0.95231164]
-----
12
1.0
[0.64108896]
-----
13
1.0
[0.7020725]
-----
14
1.0
[0.7434792]
-----
16
1.0
[0.7898997]
-----
17
1.0
[0.6997979]
-----
21
1.0
[0.68866503]
-----
23
1.0
[0.73444915]
-----
24
1.0
[0.8029383]
-----
25
1.0
[0.956305]
-----
29
1.0
[0.82491755]
-----
32
1.0
[0.07498613]
-----
33
1.0
[0.36103857]
-----
34
1.0
[0.22847599]
-----
35
1.0
[0.8699136]
-----
37
1.0
[0.82480633]
-----
39
1.0
[0.52518547]
-----
40
1.0
[0.7120949]
-----
42
1.0
[0.943895]
-----
43
1.0
[0.97172475]
-----
44
1.0
[0.95956516]
-----
48
1.0
[0.27687278]
-----
50
1.0
[0.7887188]
-----
51
1.0
[0.5479308]
-----
52
1.0
[0.8517644]
-----
55
1.0
[0.63482666]
-----
59
1.0
[0.49442732]
-----
62
1.0
[0.82599425]
-----
64
1.0
[0.9608758]
-----
67
1.0
[0.68656605]
-----
68
1.0
[0.84160894]
-----
70
1.0
[0.81651986]
-----
71
1.0
[0.8564406]
-----
73
1.0
[0.7271982]
-----
74
1.0
[0.9413922]
-----
75
1.0
[0.8795289]
-----
76
1.0
[0.6918774]
-----
79
1.0
[